In [ ]:
import email 
import sms
import whatsapp as ws
import aws 
def alert(photos,c):
    date = datetime.datetime.now() 
    t = time.localtime()
    current_time = time.strftime("%H-%M-%S", t)
    To = "bhavnalatare@gmail.com"
    Subject = "Face Detetcted !!"
    Message = f"Encountered Timestamp : {date}"
    filename = []
    c+=1
    print(c)
    for photo in photos:
        img= f"{current_time}.jpg"
        filename.append(img)
        print(filename)
        cv2.imwrite(img,photo)

    ws.send_what(date)
    return c


In [ ]:
import cv2
import numpy as np
import time

# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Initialize Webcam
index=0
cap = cv2.VideoCapture(index)
count = 0

# Collect 400 samples of your face from webcam input
# time.sleep(50)
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = './faces/user/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")


In [ ]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

# Get the training data we previously made
data_path = './faces/user/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

vimal_model  = cv2.face_LBPHFaceRecognizer.create()
# Let's train our model 
vimal_model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully")


In [ ]:
import cv2
import numpy as np
import os
import time
import datetime
import email 
import sms
import whatsapp as ws
import aws
def alert(photos,c):
    date = datetime.datetime.now() 
    t = time.localtime()
    current_time = time.strftime("%H-%M-%S", t)
    To = "bhavnalatare@gmail.com"
    Subject = "Face Detetcted !!"
    Message = f"Encountered Timestamp : {date}"
    filename = []
    c+=1
    print(c)
    for photo in photos:
        img= f"{current_time}.jpg"
        filename.append(img)
        print(filename)
        cv2.imwrite(img,photo)
    sms.sendsms()
    email.send_email(To,Subject,Message,filename)
    aws.aws_automation()
    ws.send_what(date)
    return c


face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(img, 1.3, 5)
    print(len(faces))
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi

# Open Webcam
cap = cv2.VideoCapture(0)
photos=[]
c = 0
while True:

    ret, frame = cap.read()
    
    image, face = face_detector(frame)
  
    face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

    # Pass face to prediction model
    # "results" comprises of a tuple containing the label and the confidence value
    results = vimal_model.predict(face)
    # harry_model.predict(face)

    if results[1] < 500:

        confidence = int( 100 * (1 - (results[1])/400) )
        display_string = str(confidence) + '% Confident it is User'

    cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)

    if confidence >= 90 :
        cv2.putText(image, "Hey Bhavna", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Recognition', image )
        if c == 0:
            photos.append(image)
            print(c)
            print("In the count1")
            c = alert(photos,c)
            print(c)
        cv2.imshow('Face Recognition', image )
        cap.release()
        break
    else:
        cv2.putText(image, "I dont know, who are you", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )

        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()


In [ ]:
cap.release()